In [1]:
#import reload
from importlib import reload


import pandas as pd
import evaluation
reload(evaluation)
from evaluation import load_model
from evaluation import get_sar_indices

In [218]:
DATASET='100K_accts_EASY25'
OUTPUTDIR='/home/agnes/desktop/flib/AMLsim/outputs/'

# path=f'/home/agnes/desktop/flib/thesis_XAML/ultra_final_data/'
normal_test=pd.read_csv(f'ultra_final_data/{DATASET}/bank/test/nodes.csv')

model, traindata, testdata, feature_names, target_names=load_model('GAT',DATASET)

df=pd.read_csv(f'{OUTPUTDIR}{DATASET}/tx_log.csv')


In [219]:
df['modelType'].unique()


array([11,  0,  3,  4,  2,  1, 10,  9,  8,  7,  5])

In [220]:
df_net = df
df_net = df_net[df_net['bankOrig'] != 'source'] 
df_net = df_net[df_net['bankDest'] != 'sink'] 
df_net = df_net[df_net['step']>=37]
#use only the sar transactions 
df_net=df_net[df_net['isSAR']==1]
df_net


,step,type,amount,nameOrig,bankOrig,daysInBankOrig,phoneChangesOrig,oldbalanceOrig,newbalanceOrig,nameDest,bankDest,daysInBankDest,phoneChangesDest,oldbalanceDest,newbalanceDest,isSAR,alertID,modelType
2979095,37,TRANSFER,1030.44,91384,bank,37,0,95693.33,94662.89,149,bank,38,0,53236.35,54266.80,1,2933,2
2982062,37,TRANSFER,2737.67,78041,bank,38,0,93018.03,90280.36,26736,bank,37,0,171148.37,173886.06,1,147,1
2982132,37,TRANSFER,1058.04,94088,bank,38,0,133949.48,132891.43,93446,bank,37,0,178874.78,179932.82,1,1950,5
2982607,37,TRANSFER,2020.67,39822,bank,37,0,52331.73,50311.06,83034,bank,38,0,0.00,0.00,1,2587,8
2985525,37,TRANSFER,1157.84,71483,bank,37,0,167397.84,166240.00,38369,bank,37,0,80607.54,81765.39,1,1226,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26389714,366,TRANSFER,622.78,94059,bank,367,0,361169.56,360546.78,69804,bank,367,0,392545.96,393168.75,1,2744,8
26392331,366,TRANSFER,1033.11,24914,bank,367,0,158065.78,157032.67,18137,bank,367,0,295333.90,296367.03,1,3972,3
26396856,366,TRANSFER,671.64,59130,bank,367,0,318589.78,317918.12,29470,bank,367,0,344027.59,344699.25,1,1002,3
26396857,366,TRANSFER,604.48,29470,bank,367,0,344699.25,344094.75,3479,bank,367,0,307322.78,307927.28,1,1002,3


In [221]:
df_net.head()

,step,type,amount,nameOrig,bankOrig,daysInBankOrig,phoneChangesOrig,oldbalanceOrig,newbalanceOrig,nameDest,bankDest,daysInBankDest,phoneChangesDest,oldbalanceDest,newbalanceDest,isSAR,alertID,modelType
2979095,37,TRANSFER,1030.44,91384,bank,37,0,95693.33,94662.89,149,bank,38,0,53236.35,54266.80,1,2933,2
2982062,37,TRANSFER,2737.67,78041,bank,38,0,93018.03,90280.36,26736,bank,37,0,171148.37,173886.06,1,147,1
2982132,37,TRANSFER,1058.04,94088,bank,38,0,133949.48,132891.43,93446,bank,37,0,178874.78,179932.82,1,1950,5
2982607,37,TRANSFER,2020.67,39822,bank,37,0,52331.73,50311.06,83034,bank,38,0,0.00,0.00,1,2587,8
2985525,37,TRANSFER,1157.84,71483,bank,37,0,167397.84,166240.00,38369,bank,37,0,80607.54,81765.39,1,1226,4


In [222]:
gb=df_net.groupby('nameOrig')
modeltypes1=gb['modelType'].mean()
modeltypes1[modeltypes1==3]

nameOrig
8        3.0
79       3.0
104      3.0
126      3.0
152      3.0
        ... 
99865    3.0
99885    3.0
99919    3.0
99950    3.0
99976    3.0
Name: modelType, Length: 3998, dtype: float64

In [223]:
gb2=df_net.groupby('nameDest')
modeltypes2=gb2['modelType'].mean()
modeltypes2[modeltypes2==3]

nameDest
8        3.0
79       3.0
104      3.0
126      3.0
152      3.0
        ... 
99865    3.0
99885    3.0
99919    3.0
99950    3.0
99976    3.0
Name: modelType, Length: 3998, dtype: float64

In [224]:
modeltypes=pd.merge(modeltypes1,modeltypes2,how='outer',left_index=True,right_index=True)

In [225]:
#merge column _x and column _y into one column
modeltypes['modelType']=modeltypes['modelType_x'].combine_first(modeltypes['modelType_y'])

In [226]:
modeltypes.drop(['modelType_x','modelType_y'],axis=1,inplace=True)


In [227]:
modeltypes[modeltypes['modelType']==3]

,modelType
8,3.0
79,3.0
104,3.0
126,3.0
152,3.0
...,...
99865,3.0
99885,3.0
99919,3.0
99950,3.0


In [228]:
#set idx column to 'account' in modeltypes
modeltypes.index.name='account'

In [229]:
#count each modeeltype
modeltypes['modelType'].value_counts()

3.0    4284
1.0    4211
2.0    4054
4.0    2776
5.0    2768
7.0    2365
8.0    2313
Name: modelType, dtype: int64

In [230]:
sar_indices_used=get_sar_indices(model,testdata)

In [231]:
predicted=normal_test.iloc[sar_indices_used]

In [232]:
#use normal_test to map the account to the node
modelframe=pd.merge(predicted,modeltypes,how='left',left_on='account',right_on='account')

In [233]:
modelframe
#keep only the is_sar column and the modelType column
modelframe=modelframe[['account','is_sar','modelType']]
modelframe



,account,is_sar,modelType
0,16248,0.0,NaN
1,98907,1.0,7.0
2,6990,1.0,8.0
3,30271,1.0,2.0
4,13321,1.0,3.0
...,...,...,...
26727,88489,0.0,NaN
26728,70887,1.0,8.0
26729,31596,1.0,3.0
26730,64836,1.0,1.0


In [234]:
#count the number of each modeltype and store the count in a separate colunmn

counts=modelframe.drop(['is_sar'],axis=1).groupby('modelType').count()
counts.columns=['Accounts found']

counts

,Accounts found
modelType,
1.0,2814
2.0,3152
3.0,2858
4.0,2723
5.0,2643
7.0,2258
8.0,1563


In [235]:
#count each modeeltype
actual_counts=modeltypes['modelType'].value_counts()
actual_counts

3.0    4284
1.0    4211
2.0    4054
4.0    2776
5.0    2768
7.0    2365
8.0    2313
Name: modelType, dtype: int64

In [236]:
#use counts and actual counts to get how many of each modeltype was predicted
counts['Total number of accounts']=actual_counts
counts['Percentage']=counts['Accounts found']/counts['Total number of accounts']
counts

,Accounts found,Total number of accounts,Percentage
modelType,,,
1.0,2814,4211,0.668250
2.0,3152,4054,0.777504
3.0,2858,4284,0.667134
4.0,2723,2776,0.980908
5.0,2643,2768,0.954841
7.0,2258,2365,0.954757
8.0,1563,2313,0.675746


In [237]:
# {"fan_out": 1, "fan_in": 2, "cycle": 3, "bipartite": 4, "stack": 5,
#                     "random": 6, "scatter_gather": 7, "gather_scatter": 8}
counts.index=counts.index.map({1:'fan_out',2:'fan_in',3:'cycle',4:'bipartite',5:'stack',6:'random',7:'scatter_gather',8:'gather_scatter'})

#put percentage column first
counts=counts[['Percentage','Accounts found','Total number of accounts']]

In [238]:
counts.sort_values('Percentage',ascending=False)

,Percentage,Accounts found,Total number of accounts
modelType,,,
bipartite,0.980908,2723,2776
stack,0.954841,2643,2768
scatter_gather,0.954757,2258,2365
fan_in,0.777504,3152,4054
gather_scatter,0.675746,1563,2313
fan_out,0.668250,2814,4211
cycle,0.667134,2858,4284


In [239]:
import torch
torch.sum(testdata.y)

tensor(22771, device='cuda:0')